In [3]:
from MGTomo import gridop 

import torch

In [4]:
# Example usage:
# Define a 31x31 input matrix
input_matrix = torch.rand(31, 31)

# Apply upsampling convolution
output_matrix = gridop.P(input_matrix)

# Check the shape of the output matrix
print("Output matrix shape:", output_matrix.shape)

Output matrix shape: torch.Size([63, 63])


In [5]:
# Example usage:
# Define a 63x63 input matrix (output from P)
input_matrix = torch.rand(63, 63)

# Apply restriction operator
output_matrix = gridop.R(input_matrix)

# Check the shape of the output matrix
print("Output matrix shape:", output_matrix.shape)

Output matrix shape: torch.Size([31, 31])


In [6]:
x = torch.randn(31,31)
y = torch.randn(63,63)

In [7]:
torch.sum(gridop.R(y)*x)

tensor(18.9666)

In [8]:
torch.sum(y*gridop.P(x))

tensor(18.9666)

In [9]:
(gridop.R(gridop.P(x)) - x).abs().min()

tensor(0.0007)

In [10]:
torch.sum(gridop.R_v3(y)*x)

tensor(-32.3416)